# पाणिनीय अष्टाध्यायी - संगणकीय व्याकरण
## Panini's Ashtadhyayi as Computational Grammar

पाणिनि (लगभग ५०० ई.पू.) ने अष्टाध्यायी में संस्कृत व्याकरण की विधिवत् एवं संगणकीय रचना प्रस्तुत की।

यह विश्व का प्रथम **formal language specification** है:

| पाणिनीय अवधारणा | आधुनिक संगणक विज्ञान |
|------------------|-----------------------|
| **माहेश्वर सूत्राणि** | Data compression, Symbol table |
| **प्रत्याहार** | Regular expressions, Character classes |
| **सूत्र** | Production rules (BNF grammar) |
| **परिभाषा** | Meta-rules, Exception handling |
| **अनुवृत्ति** | Context inheritance, Scope |
| **पुनरावृत्ति** | Recursion |
| **सिद्ध/असिद्ध** | Ordering constraints |

## विषयसूची

1. माहेश्वर सूत्राणि - Data Compression
2. प्रत्याहार निष्कर्षण - Pattern Extraction Algorithm
3. सूत्र व्यवस्था - Rule System
4. पुनरावृत्ति - Recursion in Rules
5. परिभाषा - Meta-rules & Conflict Resolution
6. अनुवृत्ति - Context Scoping
7. BNF Grammar Comparison
8. Finite Automata Representation
9. Visualizations

### Setup (केवल Google Colab पर)
```python
# !pip install plotly ipywidgets matplotlib networkx
# !git clone https://github.com/vagambhrini/dakshilipy.git
```

In [ ]:
import sys
sys.path.append("./dakshilipy/लिपी")
sys.path.append("../../लिपी")

In [ ]:
from दाक्षिलिपीहिन्दी import *
from गणितसहायकसर्गाणिहिन्दी import प्रत्याहार_निकालें

# Visualization imports
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle, FancyArrowPatch, Circle
import numpy as np

try:
    import networkx as nx
    NETWORKX_AVAILABLE = True
except ImportError:
    NETWORKX_AVAILABLE = False
    print("NetworkX not available. Install with: pip install networkx")

try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    print("Plotly not available. Using matplotlib only.")

print("Imports successful!")

---
## १. माहेश्वर सूत्राणि (Shiva Sutras)

### Data Compression का प्राचीनतम उदाहरण

पाणिनि ने संस्कृत के सभी वर्णों (phonemes) को **14 सूत्रों** में संक्षिप्त किया:

```
१. अइउण्          (स्वर - मूल)
२. ऋऌक्           (वर्ण-स्वर)
३. एओङ्           (संधि स्वर)
४. ऐऔच्           (दीर्घ संधि)
५. हयवरट्         (अन्तःस्थ)
६. लण्            (अन्तःस्थ)
७. ञमङणनम्       (नासिक्य)
८. झभञ्          (घोष महाप्राण)
९. घढधष्         (घोष महाप्राण)
१०. जबगडदश्      (घोष अल्पप्राण)
११. खफछठथचटतव्  (अघोष महाप्राण)
१२. कपय्          (अघोष अल्पप्राण)
१३. शषसर्         (ऊष्म)
१४. हल्           (सभी व्यञ्जन)
```

### Compression Ratio
- **कुल वर्ण**: 47 phonemes
- **सूत्र लम्बाई**: 14 sutras, ~48 characters (including markers)
- **इत् चिह्न**: ण्, क्, ङ्, च्, ट्, ण्, म्, ञ्, ष्, श्, व्, य्, र्, ल् (14 markers)

यह **Huffman coding** जैसी compression scheme का 2500 वर्ष पूर्व का रूप है!

In [ ]:
# १४ माहेश्वर सूत्राणि
माहेश्वरसूत्राणि = [
    'अइउण्',      # 1. a, i, u + marker
    'ऋऌक्',       # 2. ṛ, ḷ + marker
    'एओङ्',       # 3. e, o + marker
    'ऐऔच्',       # 4. ai, au + marker
    'हयवरट्',     # 5. semivowels
    'लण्',        # 6. l + marker
    'ञमङणनम्',   # 7. nasals
    'झभञ्',       # 8. voiced aspirates
    'घढधष्',      # 9. voiced aspirates
    'जबगडदश्',    # 10. voiced unaspirates
    'खफछठथचटतव्', # 11. voiceless aspirates
    'कपय्',       # 12. voiceless unaspirates
    'शषसर्',      # 13. sibilants
    'हल्'         # 14. all consonants
]

सर्ग माहेश्वरसूत्र_दर्शय():
    """
    माहेश्वर सूत्रों का प्रदर्शन।
    """
    दर्शय("=" * 50)
    दर्शय("माहेश्वर सूत्राणि (Shiva Sutras)")
    दर्शय("=" * 50)
    
    क्रमशः i, सूत्र में enumerate(माहेश्वरसूत्राणि, 1):
        # Extract marker (इत् वर्ण)
        इत् = सूत्र[-1] यदि '्' अंतर्गत सूत्र अन्यथा ''
        वर्ण = सूत्र[:-1] यदि '्' अंतर्गत सूत्र अन्यथा सूत्र
        
        दर्शय(f"{i:2}. {सूत्र:15} → वर्ण: {वर्ण}, इत्: {इत्}")
    
    दर्शय("=" * 50)

माहेश्वरसूत्र_दर्शय()

In [ ]:
सर्ग संपीडन_विश्लेषण():
    """
    Data compression analysis of Maheshvara Sutras.
    """
    # Count total phonemes
    कुल_वर्ण = 0
    क्रमशः सूत्र में माहेश्वरसूत्राणि:
        # Remove halant marker
        वर्ण = सूत्र[:-1] यदि '्' अंतर्गत सूत्र अन्यथा सूत्र
        कुल_वर्ण += len(वर्ण)
    
    कुल_सूत्र = len(माहेश्वरसूत्राणि)
    कुल_अक्षर = sum(अध्यास(len, माहेश्वरसूत्राणि))
    
    दर्शय("\n" + "=" * 50)
    दर्शय("Compression Analysis")
    दर्शय("=" * 50)
    दर्शय(f"कुल वर्ण (phonemes): {कुल_वर्ण}")
    दर्शय(f"कुल सूत्र: {कुल_सूत्र}")
    दर्शय(f"कुल अक्षर (with markers): {कुल_अक्षर}")
    दर्शय(f"संपीडन अनुपात: {कुल_वर्ण}/{कुल_अक्षर} = {कुल_वर्ण/कुल_अक्षर:.2f}")
    दर्शय("\nआधुनिक समतुल्य: Huffman coding, Symbol table")
    दर्शय("=" * 50)

संपीडन_विश्लेषण()

---
## २. प्रत्याहार (Pratyahara) - Pattern Extraction

### Algorithm for Character Class Definition

**प्रत्याहार** एक कूट संकेत है जो वर्णों के समूह को निर्दिष्ट करता है:

**विधि:**
1. **आदि वर्ण** (first letter) लें
2. **इत् चिह्न** (marker) लें
3. आदि से लेकर इत् वाले वर्ण तक सभी वर्ण निकालें (इत् को छोड़कर)

**उदाहरण:**
- **अण्** = अ से ण् तक = {अ, इ, उ, ऋ, ऌ} (all vowels)
- **अक्** = अ से क् तक = {अ, इ, उ, ऋ, ऌ} (same)
- **हल्** = ह से ल् तक = सभी व्यञ्जन (all consonants)
- **अच्** = अ से च् तक = सभी स्वर (all vowels including e, o, ai, au)

यह **regular expressions** में `[a-z]` जैसे character classes का प्राचीन रूप है!

In [ ]:
सर्ग प्रत्याहार_निष्कर्षण(आदि, अन्त, शिवसूत्राणि=अज्ञात):
    """
    प्रत्याहार extraction algorithm from Shiva Sutras.
    
    Parameters:
    -----------
    आदि : str
        Starting letter
    अन्त : str
        Ending it marker (without halant)
    शिवसूत्राणि : list
        14 Maheshvara Sutras (default: standard)
    
    Returns:
    --------
    dict : Pratyahara with extracted letters
    """
    यदि शिवसूत्राणि == अज्ञात:
        शिवसूत्राणि = माहेश्वरसूत्राणि
    
    # Build complete letter list with markers
    वर्ण_सूची = []
    इत्_सूची = []
    
    क्रमशः सूत्र में शिवसूत्राणि:
        क्रमशः i, अक्षर में enumerate(सूत्र):
            यदि अक्षर == '्':
                # This is marker indicator
                यदि i > 0:
                    # Previous letter is the it marker
                    इत्_सूची.append(सूत्र[i-1])
            अन्यथा:
                यदि i == len(सूत्र) - 1 या (i < len(सूत्र) - 1 और सूत्र[i+1] != '्'):
                    वर्ण_सूची.append(अक्षर)
    
    # Extract pratyahara
    निष्कर्ष = []
    प्रारम्भ_मिला = False
    
    क्रमशः वर्ण में वर्ण_सूची:
        यदि वर्ण == आदि:
            प्रारम्भ_मिला = True
        
        यदि प्रारम्भ_मिला:
            निष्कर्ष.append(वर्ण)
            
            # Check if this is the it marker we're looking for
            यदि वर्ण == अन्त और वर्ण अंतर्गत इत्_सूची:
                # Remove the marker itself from result
                निष्कर्ष = निष्कर्ष[:-1]
                ब्रेक
    
    सर्गफल {
        'प्रत्याहार': आदि + अन्त,
        'आदि': आदि,
        'इत्': अन्त,
        'वर्ण': ''.join(निष्कर्ष),
        'वर्ण_सूची': निष्कर्ष,
        'संख्या': len(निष्कर्ष)
    }

# Test pratyahara extraction
दर्शय("\nप्रत्याहार Examples:")
दर्शय("=" * 50)

परीक्षाएँ = [
    ('अ', 'ण्', 'Short vowels'),
    ('अ', 'क्', 'Simple vowels'),
    ('अ', 'च्', 'All vowels'),
    ('ह', 'ल्', 'All consonants'),
    ('इ', 'क्', 'i, u, ṛ, ḷ'),
    ('य', 'ण्', 'Semivowels'),
]

क्रमशः आदि, अन्त, विवरण में परीक्षाएँ:
    फल = प्रत्याहार_निष्कर्षण(आदि, अन्त)
    दर्शय(f"{फल['प्रत्याहार']:6} → {फल['वर्ण']:20} ({फल['संख्या']} letters) - {विवरण}")

In [ ]:
# Common pratyaharas used in Ashtadhyayi
सर्ग सामान्य_प्रत्याहाराः():
    """
    Display common pratyaharas used frequently in Panini's grammar.
    """
    दर्शय("\n" + "=" * 60)
    दर्शय("सामान्य प्रत्याहाराः (Common Pratyaharas in Ashtadhyayi)")
    दर्शय("=" * 60)
    
    प्रत्याहार_सूची = [
        ('अ', 'च्', 'अच्', 'All vowels'),
        ('ह', 'ल्', 'हल्', 'All consonants'),
        ('अ', 'ण्', 'अण्', 'Simple vowels (a, i, u, ṛ, ḷ)'),
        ('इ', 'क्', 'इक्', 'i, u, ṛ, ḷ'),
        ('ए', 'च्', 'एच्', 'Diphthongs (e, o, ai, au)'),
        ('य', 'ण्', 'यण्', 'Semivowels (y, v, r, l)'),
        ('झ', 'ल्', 'झल्', 'Voiced consonants'),
        ('ख', 'य्', 'खय्', 'Voiceless consonants'),
    ]
    
    क्रमशः आदि, इत्, नाम, अर्थ में प्रत्याहार_सूची:
        फल = प्रत्याहार_निष्कर्षण(आदि, इत्)
        दर्शय(f"\n{नाम:8} ({फल['संख्या']:2} letters)")
        दर्शय(f"  Meaning: {अर्थ}")
        दर्शय(f"  Letters: {फल['वर्ण']}")
    
    दर्शय("\n" + "=" * 60)

सामान्य_प्रत्याहाराः()

---
## ३. पुनरावृत्ति (Recursion) in Grammar Rules

### Recursive Rule Application

पाणिनि के सूत्र **recursive** हैं - एक नियम का परिणाम दूसरे नियम का निवेश बनता है।

**उदाहरण: समास (Compound Formation)**

```
सूत्र: समर्थः पदविधिः (२.१.१)
"सार्थक शब्द compound बना सकते हैं"

Recursion:
राज + पुत्र → राजपुत्र
राजपुत्र + सेवक → राजपुत्रसेवक
राजपुत्रसेवक + गृह → राजपुत्रसेवकगृह
...
```

यह **context-free grammar** का मूलभूत गुण है।

In [ ]:
सर्ग समास_निर्माण(पदसूची, गहराई=0, अधिकतम_गहराई=10):
    """
    Recursive compound formation (samasa).
    Demonstrates recursion in Paninian grammar.
    
    Parameters:
    -----------
    पदसूची : list
        List of words to compound
    गहराई : int
        Current recursion depth
    अधिकतम_गहराई : int
        Maximum recursion depth
    
    Returns:
    --------
    str : Compound word
    """
    # Base case
    यदि len(पदसूची) == 0:
        सर्गफल ""
    यदि len(पदसूची) == 1:
        सर्गफल पदसूची[0]
    यदि गहराई >= अधिकतम_गहराई:
        सर्गफल '...'
    
    # Recursive case: compound first two words
    प्रथम = पदसूची[0]
    द्वितीय = पदसूची[1]
    शेष = पदसूची[2:]
    
    # Simple concatenation (actual sandhi rules would apply)
    समास = प्रथम + द्वितीय
    
    दर्शय("  " * गहराई + f"Level {गहराई}: {प्रथम} + {द्वितीय} → {समास}")
    
    # Recurse with compound and remaining words
    यदि len(शेष) > 0:
        सर्गफल समास_निर्माण([समास] + शेष, गहराई + 1, अधिकतम_गहराई)
    अन्यथा:
        सर्गफल समास

# Demonstrate recursive compound formation
दर्शय("\nRecursive Compound Formation (समास):")
दर्शय("=" * 50)

शब्द = ['राज', 'पुत्र', 'सेवक', 'गृह']
दर्शय(f"Input words: {शब्द}\n")

परिणाम = समास_निर्माण(शब्द)
दर्शय(f"\nFinal compound: {परिणाम}")
दर्शय("\nThis demonstrates recursion in Paninian grammar rules.")

In [ ]:
सर्ग व्युत्पत्ति_वृक्ष(मूल, नियम_सूची, गहराई=0, अधिकतम_गहराई=5):
    """
    Derivation tree showing recursive rule application.
    
    Parameters:
    -----------
    मूल : str
        Root form
    नियम_सूची : list of tuples
        Rules to apply (name, transformation function)
    गहराई : int
        Current depth
    अधिकतम_गहराई : int
        Maximum depth
    
    Returns:
    --------
    list : Derivation steps
    """
    यदि गहराई >= अधिकतम_गहराई या len(नियम_सूची) == 0:
        सर्गफल [(मूल, 'final', गहराई)]
    
    चरण = []
    वर्तमान = मूल
    
    क्रमशः i, (नियम_नाम, नियम_सर्ग) में enumerate(नियम_सूची):
        नवीन = नियम_सर्ग(वर्तमान)
        चरण.append((वर्तमान, नियम_नाम, गहराई))
        
        यदि नवीन != वर्तमान:
            वर्तमान = नवीन
    
    चरण.append((वर्तमान, 'सिद्ध', गहराई))
    सर्गफल चरण

# Example: Verb derivation
दर्शय("\nVerb Derivation Example (क्रिया निष्पत्ति):")
दर्शय("=" * 50)

# Simple transformation rules (simplified for demonstration)
नियमाः = [
    ('धातु', समीकरण x: x + '्'),  # Add virama
    ('प्रथम-पुरुष', समीकरण x: x.replace('्', 'ति')),  # 3rd person
]

व्युत्पत्ति = व्युत्पत्ति_वृक्ष('गम', नियमाः)

क्रमशः रूप, नियम, स्तर में व्युत्पत्ति:
    दर्शय("  " * स्तर + f"{रूप} [{नियम}]")

दर्शय("\nThis shows step-by-step derivation via rule application.")

---
## ४. परिभाषा (Meta-rules) - Conflict Resolution

### Meta-linguistic Rules for Grammar Interpretation

पाणिनि के **परिभाषा सूत्र** grammar के interpretation को नियंत्रित करते हैं।

ये **compiler directives** या **pragma statements** के समान हैं।

### प्रमुख परिभाषा सूत्र:

| सूत्र | अर्थ | CS Equivalent |
|-------|------|---------------|
| **विप्रतिषेधे परं कार्यम्** (१.४.२) | "संघर्ष में, बाद वाला नियम लागू हो" | Last rule wins |
| **अन्तरङ्गं बहिरङ्गाच्च** | "आंतरिक नियम बाहरी से पहले" | Priority ordering |
| **असिद्धवत्** (६.४.२२) | "कुछ नियम अन्य के लिए 'अदृश्य' हैं" | Scope isolation |
| **पूर्वत्रासिद्धम्** (८.२.१) | "पूर्व चरण बाद के लिए असिद्ध" | Phase ordering |

ये नियम **precedence rules** और **exception handling** को formalize करते हैं।

In [ ]:
सर्ग नियम_संघर्ष_समाधान(निवेश, नियम_सूची, समाधान_विधि='बाद_वाला'):
    """
    Conflict resolution among competing rules.
    Implements: विप्रतिषेधे परं कार्यम् (last rule wins).
    
    Parameters:
    -----------
    निवेश : str
        Input string
    नियम_सूची : list of tuples
        Rules (name, condition, transformation)
    समाधान_विधि : str
        'बाद_वाला' (last), 'प्राथमिकता' (priority)
    
    Returns:
    --------
    tuple : (result, applied_rule)
    """
    लागू_नियम = []
    
    # Find all applicable rules
    क्रमशः नियम_नाम, शर्त, रूपान्तरण में नियम_सूची:
        यदि शर्त(निवेश):
            लागू_नियम.append((नियम_नाम, रूपान्तरण))
    
    यदि len(लागू_नियम) == 0:
        सर्गफल (निवेश, 'कोई नहीं')
    
    यदि len(लागू_नियम) == 1:
        नाम, रूपान्तरण = लागू_नियम[0]
        सर्गफल (रूपान्तरण(निवेश), नाम)
    
    # Multiple rules apply - conflict!
    दर्शय(f"\n⚠️  Rule conflict detected! {len(लागू_नियम)} rules applicable:")
    क्रमशः नाम, _ में लागू_नियम:
        दर्शय(f"  - {नाम}")
    
    यदि समाधान_विधि == 'बाद_वाला':
        # विप्रतिषेधे परं कार्यम् - Last rule wins
        नाम, रूपान्तरण = लागू_नियम[-1]
        दर्शय(f"\n✓ Resolution: 'बाद_वाला' (last) → Applying '{नाम}'")
        सर्गफल (रूपान्तरण(निवेश), नाम)
    
    अन्यथा:
        # Default: first rule
        नाम, रूपान्तरण = लागू_नियम[0]
        दर्शय(f"\n✓ Resolution: 'प्रथम' (first) → Applying '{नाम}'")
        सर्गफल (रूपान्तरण(निवेश), नाम)

# Example: Conflicting sandhi rules
दर्शय("\nConflict Resolution Example (विप्रतिषेध समाधान):")
दर्शय("=" * 60)

# Simplified sandhi rules (for demonstration)
संधि_नियमाः = [
    ('गुण-संधि', समीकरण x: 'अ' अंतर्गत x, समीकरण x: x + '_guna'),
    ('वृद्धि-संधि', समीकरण x: 'अ' अंतर्गत x, समीकरण x: x + '_vriddhi'),
    ('दीर्घ-संधि', समीकरण x: True, समीकरण x: x + '_deergh'),
]

निवेश = 'अ+इ'
दर्शय(f"Input: {निवेश}\n")

परिणाम, नियम = नियम_संघर्ष_समाधान(निवेश, संधि_नियमाः, 'बाद_वाला')
दर्शय(f"\nResult: {परिणाम}")
दर्शय(f"Applied rule: {नियम}")
दर्शय("\nThis implements: विप्रतिषेधे परं कार्यम् (१.४.२)")

---
## ५. अनुवृत्ति (Context Inheritance)

### Scoping and Context Propagation

**अनुवृत्ति** = एक सूत्र से शब्द अगले सूत्रों में स्वतः जाते हैं।

यह **lexical scoping** और **context passing** का प्राचीन उदाहरण है।

**उदाहरण:**
```
सूत्र ७.१.१: स्वौजसमौट्... (विभक्ति प्रत्यय)
सूत्र ७.१.२: [स्वौजसमौट्...] गुणः  (अनुवृत्ति from 7.1.1)
सूत्र ७.१.३: [स्वौजसमौट्...] [गुणः] अतः  (from 7.1.1 and 7.1.2)
```

यह code में **variable scope** के समान है:
```python
# Outer scope
def rule_7_1_1():
    context = 'svaujas...'  # Defined here
    
    def rule_7_1_2():
        # context available here (anuvritti)
        use(context)
```

In [ ]:
सर्ग अनुवृत्ति_प्रदर्शन(सूत्र_सूची):
    """
    Demonstrate anuvritti (context inheritance) in sutras.
    
    Parameters:
    -----------
    सूत्र_सूची : list of tuples
        (sutra_number, new_content, inherited_from)
    
    Returns:
    --------
    list : Full sutras with context
    """
    संदर्भ_स्टैक = []  # Context stack (like scope chain)
    पूर्ण_सूत्र = []
    
    दर्शय("\nAnuvritti (Context Inheritance) Demonstration:")
    दर्शय("=" * 60)
    
    क्रमशः सूत्र_संख्या, नवीन_अंश, विरासत_स्तर में सूत्र_सूची:
        # Manage context stack
        यदि विरासत_स्तर < len(संदर्भ_स्टैक):
            # Pop contexts that are out of scope
            संदर्भ_स्टैक = संदर्भ_स्टैक[:विरासत_स्तर]
        
        # Add new content to context
        संदर्भ_स्टैक.append(नवीन_अंश)
        
        # Build full sutra with inherited context
        पूर्ण = ' '.join(संदर्भ_स्टैक)
        पूर्ण_सूत्र.append((सूत्र_संख्या, पूर्ण))
        
        # Display
        विरासत_चिह्न = f" [← {विरासत_स्तर} contexts]" यदि विरासत_स्तर > 0 अन्यथा ""
        दर्शय(f"\n{सूत्र_संख्या}: {नवीन_अंश}{विरासत_चिह्न}")
        दर्शय(f"  Full: {पूर्ण}")
        दर्शय(f"  Context stack: {संदर्भ_स्टैक}")
    
    दर्शय("\n" + "=" * 60)
    सर्गफल पूर्ण_सूत्र

# Example: Series of sutras with anuvritti
सूत्र_उदाहरण = [
    ('७.१.१', 'युष्मद्युपपदे', 0),  # Base context
    ('७.१.२', 'सम्बुद्धौ', 1),       # Inherits from 7.1.1
    ('७.१.३', 'अतः', 2),             # Inherits from 7.1.1 and 7.1.2
    ('७.१.४', 'ङित्', 1),            # Back to inheriting only from 7.1.1
]

परिणाम = अनुवृत्ति_प्रदर्शन(सूत्र_उदाहरण)

दर्शय("\nThis demonstrates lexical scoping in Paninian grammar!")

---
## ६. BNF Grammar Comparison

### Panini's System vs. Backus-Naur Form

पाणिनीय व्याकरण और BNF (1959) में आश्चर्यजनक समानता:

| Panini (500 BCE) | BNF (1959 CE) | Example |
|------------------|---------------|----------|
| **सूत्र** | Production rule | `<noun> ::= <stem> <ending>` |
| **प्रत्याहार** | Terminal class | `<vowel> ::= a \| i \| u` |
| **विकल्प (वा)** | Alternative (\|) | `a \| b` |
| **पुनरावृत्ति** | Recursion | `<expr> ::= <expr> + <expr>` |
| **सिद्ध** | Derived form | Terminal output |

**BNF Example:**
```bnf
<sentence> ::= <subject> <verb> <object>
<subject>  ::= <noun> | <pronoun>
<noun>     ::= <stem> <case-ending>
```

**Paninian Equivalent:**
```
वाक्यम् = कर्ता क्रिया कर्म
कर्ता = नाम वा सर्वनाम
नाम = धातु विभक्ति
```

In [ ]:
सर्ग पाणिनीय_से_BNF(पाणिनि_नियम):
    """
    Convert Paninian-style rule to BNF notation.
    
    Parameters:
    -----------
    पाणिनि_नियम : dict
        {'नाम': str, 'बायाँ': str, 'दायाँ': list}
    
    Returns:
    --------
    str : BNF rule
    """
    नाम = पाणिनि_नियम['नाम']
    बायाँ = पाणिनि_नियम['बायाँ']
    दायाँ = पाणिनि_नियम['दायाँ']
    
    # Convert alternatives
    विकल्प = ' | '.join(दायाँ)
    
    BNF = f"<{बायाँ}> ::= {विकल्प}"
    सर्गफल BNF

सर्ग BNF_तुलना_प्रदर्शन():
    """
    Show comparison between Paninian and BNF grammar notation.
    """
    दर्शय("\nPaninian Grammar vs. BNF Comparison:")
    दर्शय("=" * 70)
    
    नियम_सूची = [
        {
            'नाम': 'Vowel definition',
            'पाणिनि': 'अच् = अ इ उ ऋ ऌ ए ओ ऐ औ (प्रत्याहार)',
            'BNF': '<vowel> ::= a | i | u | ṛ | ḷ | e | o | ai | au'
        },
        {
            'नाम': 'Noun formation',
            'पाणिनि': 'नाम = प्रातिपदिक + सुप् (सूत्र ४.१.२)',
            'BNF': '<noun> ::= <stem> <case-ending>'
        },
        {
            'नाम': 'Compound (recursive)',
            'पाणिनि': 'समास = पद + पद वा समास + पद (पुनरावृत्ति)',
            'BNF': '<compound> ::= <word> <word> | <compound> <word>'
        },
        {
            'नाम': 'Sentence structure',
            'पाणिनि': 'वाक्य = कर्ता + क्रिया + कर्म',
            'BNF': '<sentence> ::= <subject> <verb> <object>'
        },
    ]
    
    क्रमशः नियम में नियम_सूची:
        दर्शय(f"\n{नियम['नाम']}:")
        दर्शय(f"  Panini:  {नियम['पाणिनि']}")
        दर्शय(f"  BNF:     {नियम['BNF']}")
    
    दर्शय("\n" + "=" * 70)
    दर्शय("\nKey Insight: Panini's system (500 BCE) predates BNF (1959 CE)")
    दर्शय("by approximately 2500 years!")
    दर्शय("=" * 70)

BNF_तुलना_प्रदर्शन()

---
## ७. Finite Automata Representation

### State Machines in Sanskrit Grammar

कुछ पाणिनीय नियम **finite automata** के रूप में represent किए जा सकते हैं।

**उदाहरण: संधि नियम (Sandhi Rules)**

```
State Machine for a + i → e:

     'अ'        'इ'
  ┌─────┐    ┌─────┐
  │  q₀ │───→│  q₁ │───→ [ए]
  └─────┘    └─────┘
   start      final
```

यह **deterministic finite automaton (DFA)** है।

In [ ]:
वर्ग संधि_स्वचालित:
    """
    Finite automaton for sandhi (phonetic) rules.
    Represents vowel sandhi as a state machine.
    """
    
    सर्ग __init__(स्व):
        # States
        स्व.अवस्थाएँ = {'q0', 'q1', 'q2', 'अस्वीकार', 'स्वीकार'}
        स्व.प्रारम्भ = 'q0'
        स्व.अन्तिम = {'स्वीकार'}
        स्व.वर्तमान = स्व.प्रारम्भ
        
        # Transition function: (state, input) → (new_state, output)
        स्व.संक्रमण = {
            ('q0', 'अ'): ('q1', अज्ञात),
            ('q1', 'इ'): ('स्वीकार', 'ए'),
            ('q1', 'उ'): ('स्वीकार', 'ओ'),
            ('q0', 'आ'): ('q2', अज्ञात),
            ('q2', 'इ'): ('स्वीकार', 'ऐ'),
            ('q2', 'उ'): ('स्वीकार', 'औ'),
        }
    
    सर्ग संक्रमण_करें(स्व, निवेश):
        """
        Process input and transition to new state.
        
        Parameters:
        -----------
        निवेश : str
            Input symbol
        
        Returns:
        --------
        str or None : Output (if any)
        """
        कुञ्जी = (स्व.वर्तमान, निवेश)
        
        यदि कुञ्जी अंतर्गत स्व.संक्रमण:
            नवीन_अवस्था, निर्गम = स्व.संक्रमण[कुञ्जी]
            दर्शय(f"  {स्व.वर्तमान} --[{निवेश}]--> {नवीन_अवस्था}" + 
                  (f"  output: {निर्गम}" यदि निर्गम अन्यथा ""))
            स्व.वर्तमान = नवीन_अवस्था
            सर्गफल निर्गम
        अन्यथा:
            दर्शय(f"  {स्व.वर्तमान} --[{निवेश}]--> अस्वीकार (invalid)")
            स्व.वर्तमान = 'अस्वीकार'
            सर्गफल अज्ञात
    
    सर्ग प्रक्रिया(स्व, निवेश_श्रृंखला):
        """
        Process a sequence of inputs.
        
        Parameters:
        -----------
        निवेश_श्रृंखला : list
            Sequence of input symbols
        
        Returns:
        --------
        str or None : Final output
        """
        स्व.वर्तमान = स्व.प्रारम्भ
        निर्गम = अज्ञात
        
        दर्शय(f"\nProcessing: {निवेश_श्रृंखला}")
        दर्शय(f"Start state: {स्व.वर्तमान}\n")
        
        क्रमशः प्रतीक में निवेश_श्रृंखला:
            परिणाम = स्व.संक्रमण_करें(प्रतीक)
            यदि परिणाम != अज्ञात:
                निर्गम = परिणाम
        
        दर्शय(f"\nFinal state: {स्व.वर्तमान}")
        दर्शय(f"Accepted: {स्व.वर्तमान अंतर्गत स्व.अन्तिम}")
        
        सर्गफल निर्गम

# Demonstrate finite automaton
दर्शय("\nFinite Automaton for Sandhi Rules:")
दर्शय("=" * 50)

स्वचालित = संधि_स्वचालित()

# Test cases
परीक्षण = [
    ['अ', 'इ'],  # अ + इ → ए (guna sandhi)
    ['अ', 'उ'],  # अ + उ → ओ (guna sandhi)
    ['आ', 'इ'],  # आ + इ → ऐ (vriddhi sandhi)
    ['आ', 'उ'],  # आ + उ → औ (vriddhi sandhi)
]

क्रमशः परीक्षा में परीक्षण:
    परिणाम = स्वचालित.प्रक्रिया(परीक्षा)
    यदि परिणाम:
        दर्शय(f"Result: {' + '.join(परीक्षा)} → {परिणाम}")
    दर्शय("-" * 50)

---
## ८. Visualizations

### Visual Representations of Grammar Concepts

In [ ]:
सर्ग शिवसूत्र_आव्यूह_दर्शय():
    """
    Visualize Shiva Sutras as a matrix showing letters and markers.
    """
    fig, ax = plt.subplots(figsize=(14, 8))
    
    y_pos = 13
    
    क्रमशः i, सूत्र में enumerate(माहेश्वरसूत्राणि):
        # Extract letters and marker
        वर्ण = []
        इत् = ''
        
        क्रमशः j, अक्षर में enumerate(सूत्र):
            यदि अक्षर == '्':
                यदि j > 0:
                    इत् = सूत्र[j-1]
            अन्यथा:
                यदि j == len(सूत्र) - 1 या (j < len(सूत्र) - 1 और सूत्र[j+1] != '्'):
                    वर्ण.append(अक्षर)
        
        # Draw sutra number
        ax.text(-0.5, y_pos, f"{i+1:2}", fontsize=12, va='center', weight='bold')
        
        # Draw letters
        क्रमशः j, व in enumerate(वर्ण):
            ax.add_patch(Rectangle((j, y_pos - 0.4), 0.8, 0.8,
                                  facecolor='lightblue', edgecolor='blue', linewidth=1.5))
            ax.text(j + 0.4, y_pos, व, fontsize=14, ha='center', va='center', weight='bold')
        
        # Draw marker
        यदि इत्:
            x_marker = len(वर्ण)
            ax.add_patch(Rectangle((x_marker, y_pos - 0.4), 0.8, 0.8,
                                  facecolor='orange', edgecolor='red', linewidth=2))
            ax.text(x_marker + 0.4, y_pos, इत् + '्', fontsize=14, 
                   ha='center', va='center', weight='bold', color='red')
            ax.text(x_marker + 1.2, y_pos, '(इत्)', fontsize=9, 
                   ha='left', va='center', style='italic', color='red')
        
        y_pos -= 1
    
    ax.set_xlim(-1, 13)
    ax.set_ylim(-1, 15)
    ax.axis('off')
    ax.set_title('माहेश्वर सूत्राणि - Matrix View\n' + 
                'Blue: Letters (वर्ण), Orange: Markers (इत्)', 
                fontsize=16, weight='bold', pad=20)
    
    plt.tight_layout()
    सर्गफल fig

fig = शिवसूत्र_आव्यूह_दर्शय()
plt.show()

In [ ]:
सर्ग प्रत्याहार_दृश्य(आदि, इत्):
    """
    Visualize pratyahara extraction process.
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Get pratyahara
    फल = प्रत्याहार_निष्कर्षण(आदि, इत्)
    
    # Flatten all letters from sutras
    सभी_वर्ण = []
    इत्_स्थान = {}
    
    क्रमशः सूत्र में माहेश्वरसूत्राणि:
        क्रमशः j, अक्षर में enumerate(सूत्र):
            यदि अक्षर == '्':
                यदि j > 0:
                    इत्_स्थान[len(सभी_वर्ण) - 1] = सूत्र[j-1]
            अन्यथा:
                यदि j == len(सूत्र) - 1 या (j < len(सूत्र) - 1 और सूत्र[j+1] != '्'):
                    सभी_वर्ण.append(अक्षर)
    
    # Find positions
    आदि_स्थान = अज्ञात
    अन्त_स्थान = अज्ञात
    
    क्रमशः i, व in enumerate(सभी_वर्ण):
        यदि व == आदि और आदि_स्थान == अज्ञात:
            आदि_स्थान = i
        यदि i अंतर्गत इत्_स्थान और इत्_स्थान[i] == इत् और अन्त_स्थान == अज्ञात:
            अन्त_स्थान = i
    
    # Draw all letters
    क्रमशः i, व में enumerate(सभी_वर्ण):
        x = i * 0.8
        
        # Determine color
        यदि आदि_स्थान <= i <= अन्त_स्थान:
            रंग = 'lightgreen' यदि i != अन्त_स्थान अन्यथा 'orange'
            ax.add_patch(Rectangle((x, 0.5), 0.7, 0.8,
                                  facecolor=रंग, edgecolor='green', linewidth=2))
        अन्यथा:
            ax.add_patch(Rectangle((x, 0.5), 0.7, 0.8,
                                  facecolor='lightgray', edgecolor='gray', linewidth=1))
        
        ax.text(x + 0.35, 0.9, व, fontsize=12, ha='center', va='center', weight='bold')
        
        # Mark it letters
        यदि i अंतर्गत इत्_स्थान:
            ax.text(x + 0.35, 0.3, '(इत्)', fontsize=8, ha='center', 
                   va='center', color='red', style='italic')
    
    # Add arrows
    यदि आदि_स्थान != अज्ञात:
        ax.annotate('', xy=(आदि_स्थान * 0.8 + 0.35, 0.4), 
                   xytext=(आदि_स्थान * 0.8 + 0.35, -0.2),
                   arrowprops=dict(arrowstyle='->', color='blue', lw=2))
        ax.text(आदि_स्थान * 0.8 + 0.35, -0.4, 'आदि', 
               fontsize=10, ha='center', color='blue', weight='bold')
    
    यदि अन्त_स्थान != अज्ञात:
        ax.annotate('', xy=(अन्त_स्थान * 0.8 + 0.35, 0.4), 
                   xytext=(अन्त_स्थान * 0.8 + 0.35, -0.2),
                   arrowprops=dict(arrowstyle='->', color='red', lw=2))
        ax.text(अन्त_स्थान * 0.8 + 0.35, -0.4, 'इत्', 
               fontsize=10, ha='center', color='red', weight='bold')
    
    ax.set_xlim(-0.5, len(सभी_वर्ण) * 0.8)
    ax.set_ylim(-0.8, 2)
    ax.axis('off')
    
    शीर्षक = f"प्रत्याहार: {फल['प्रत्याहार']} → {फल['वर्ण']} ({फल['संख्या']} letters)"
    ax.set_title(शीर्षक, fontsize=14, weight='bold', pad=20)
    
    # Legend
    ax.text(0, 1.8, 'Green: Included | Orange: Marker (इत्) | Gray: Excluded', 
           fontsize=10, style='italic')
    
    plt.tight_layout()
    सर्गफल fig

# Visualize common pratyaharas
दर्शय("\nPratyahara Visualizations:")
दर्शय("=" * 50)

उदाहरण = [('अ', 'च्'), ('ह', 'ल्'), ('य', 'ण्')]

क्रमशः आदि, इत् में उदाहरण:
    fig = प्रत्याहार_दृश्य(आदि, इत्)
    plt.show()
    दर्शय("")

In [ ]:
सर्ग नियम_वृक्ष_दर्शय(मूल, परिवर्तन_सूची):
    """
    Visualize rule application as a tree.
    Shows derivation path from root to final form.
    
    Parameters:
    -----------
    मूल : str
        Root form
    परिवर्तन_सूची : list of tuples
        (rule_name, transform_function)
    """
    यदि not NETWORKX_AVAILABLE:
        दर्शय("NetworkX not available. Install: pip install networkx")
        सर्गफल अज्ञात
    
    # Build derivation graph
    G = nx.DiGraph()
    
    वर्तमान = मूल
    नोड_संख्या = 0
    
    # Add root node
    G.add_node(नोड_संख्या, रूप=वर्तमान, नियम='मूल', स्तर=0)
    पूर्व_नोड = नोड_संख्या
    नोड_संख्या += 1
    
    # Apply transformations
    क्रमशः नियम_नाम, परिवर्तन में परिवर्तन_सूची:
        नवीन = परिवर्तन(वर्तमान)
        G.add_node(नोड_संख्या, रूप=नवीन, नियम=नियम_नाम, स्तर=नोड_संख्या)
        G.add_edge(पूर्व_नोड, नोड_संख्या, नियम=नियम_नाम)
        पूर्व_नोड = नोड_संख्या
        नोड_संख्या += 1
        वर्तमान = नवीन
    
    # Draw graph
    fig, ax = plt.subplots(figsize=(12, 8))
    
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    # Arrange nodes vertically by level
    क्रमशः node में G.nodes():
        स्तर = G.nodes[node]['स्तर']
        pos[node] = (0, -स्तर * 2)
    
    # Draw nodes
    node_colors = ['lightgreen' यदि node == 0 अन्यथा 
                   'lightcoral' यदि node == len(G.nodes()) - 1 अन्यथा 'lightblue'
                   for node अंतर्गत G.nodes()]
    
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                          node_size=3000, ax=ax)
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, edge_color='gray', 
                          arrows=True, arrowsize=20, 
                          arrowstyle='->', ax=ax, width=2)
    
    # Draw labels
    labels = {node: G.nodes[node]['रूप'] for node अंतर्गत G.nodes()}
    nx.draw_networkx_labels(G, pos, labels, font_size=12, 
                           font_weight='bold', ax=ax)
    
    # Draw edge labels (rules)
    edge_labels = nx.get_edge_attributes(G, 'नियम')
    nx.draw_networkx_edge_labels(G, pos, edge_labels, 
                                font_size=9, font_color='red', ax=ax)
    
    ax.set_title(f'Rule Application Tree\nDerivation: {मूल} → {वर्तमान}', 
                fontsize=14, weight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    सर्गफल fig

# Example: Simple derivation
यदि NETWORKX_AVAILABLE:
    दर्शय("\nRule Application Tree:")
    दर्शय("=" * 50)
    
    नियम = [
        ('Add suffix', समीकरण x: x + 'ति'),
        ('Sandhi', समीकरण x: x.replace('अति', 'आति')),
        ('Final', समीकरण x: x),
    ]
    
    fig = नियम_वृक्ष_दर्शय('गच्छ', नियम)
    plt.show()
अन्यथा:
    दर्शय("Install networkx for tree visualization: pip install networkx")

---
## ९. Computational Complexity

### Algorithmic Analysis of Paninian Grammar

**प्रत्याहार Algorithm Complexity:**
- Time: O(n) where n = total letters in Shiva Sutras (~47)
- Space: O(1) - fixed size
- यह एक **linear scan** algorithm है

**Rule Application Complexity:**
- Best case: O(1) - direct match
- Worst case: O(n×m) where n = rules, m = string length
- पाणिनि ने ordering constraints से यह optimize किया

**Anuvritti (Context Inheritance):**
- Time: O(1) lookup via context stack
- Space: O(d) where d = max context depth (~10)
- Modern equivalent: **symbol table** with scoping

### Modern Applications

1. **Natural Language Processing (NLP)**
   - Morphological analyzers for Sanskrit
   - Sandhi splitting algorithms

2. **Compiler Design**
   - Lexical analysis (pratyahara = character classes)
   - Parser generation (sutras = grammar rules)

3. **Formal Language Theory**
   - Context-sensitive grammars
   - Chomsky hierarchy classification

In [ ]:
import time

सर्ग प्रत्याहार_जटिलता_विश्लेषण():
    """
    Analyze time complexity of pratyahara extraction.
    """
    दर्शय("\nComputational Complexity Analysis:")
    दर्शय("=" * 60)
    
    # Test pratyahara extraction speed
    परीक्षण = [
        ('अ', 'च्'),
        ('ह', 'ल्'),
        ('य', 'ण्'),
    ]
    
    पुनरावृत्ति = 10000
    
    क्रमशः आदि, इत् में परीक्षण:
        प्रारम्भ = time.time()
        
        क्रमशः _ में range(पुनरावृत्ति):
            फल = प्रत्याहार_निष्कर्षण(आदि, इत्)
        
        अन्त = time.time()
        कुल_समय = अन्त - प्रारम्भ
        प्रति_कॉल = (कुल_समय / पुनरावृत्ति) * 1000000  # microseconds
        
        दर्शय(f"\nPratyahara: {आदि}{इत्}")
        दर्शय(f"  Total time: {कुल_समय:.4f} seconds ({पुनरावृत्ति} iterations)")
        दर्शय(f"  Per call: {प्रति_कॉल:.2f} microseconds")
        दर्शय(f"  Complexity: O(n) where n = {len(फल['वर्ण_सूची'])} letters")
    
    दर्शय("\n" + "=" * 60)
    दर्शय("Conclusion: Pratyahara extraction is extremely efficient!")
    दर्शय("Linear time complexity with minimal constant factor.")
    दर्शय("=" * 60)

प्रत्याहार_जटिलता_विश्लेषण()

---
## सारांश (Summary)

### पाणिनीय व्याकरण की संगणकीय उपलब्धियाँ:

1. **Data Compression** (माहेश्वर सूत्र)
   - 47 phonemes → 14 sutras with markers
   - Efficient symbol table representation

2. **Pattern Matching** (प्रत्याहार)
   - Regular expression-like character classes
   - O(n) extraction algorithm

3. **Recursion** (पुनरावृत्ति)
   - Recursive rule application
   - Context-free grammar properties

4. **Meta-rules** (परिभाषा)
   - Conflict resolution strategies
   - Exception handling mechanisms

5. **Context Management** (अनुवृत्ति)
   - Lexical scoping
   - Context stack implementation

6. **Formal Specification**
   - BNF-like production rules (2500 years before BNF!)
   - Finite automata representation

### आधुनिक प्रभाव:

- Influenced Leonard Bloomfield (structural linguistics)
- Inspired Backus-Naur Form designers
- Foundation for Sanskrit computational tools
- Model for natural language processing

### प्रमुख सूत्र:

```
सिद्धे शब्दार्थसंबन्धः (१.२.६४)
"In a correctly derived form, meaning is established."

This is the computational linguistics principle:
Syntax + Semantics = Meaning
```

---
## अभ्यास प्रश्न (Exercises)

1. **प्रत्याहार निष्कर्षण:**
   - Extract 'इक्' pratyahara. What letters does it contain?
   - Find a pratyahara that includes only consonants.

2. **Compression Analysis:**
   - Calculate the exact compression ratio of Maheshvara Sutras.
   - Compare with modern compression algorithms (Huffman, LZW).

3. **BNF Grammar:**
   - Write BNF rules for Sanskrit case endings (विभक्ति).
   - Convert a Paninian sutra to BNF notation.

4. **Finite Automaton:**
   - Design a DFA for a simple sandhi rule.
   - Implement state transitions in code.

5. **Recursion:**
   - Implement a recursive समास (compound) generator.
   - Trace the recursion tree for a 5-word compound.

6. **Modern Applications:**
   - Research Sanskrit morphological analyzers (e.g., Heritage platform).
   - Explore how Paninian grammar influences NLP tools.

In [ ]:
# Exercise solutions
दर्शय("\nExercise Solutions:")
दर्शय("=" * 60)

# 1. Extract 'इक्' pratyahara
इक् = प्रत्याहार_निष्कर्षण('इ', 'क्')
दर्शय(f"\n1. इक् pratyahara: {इक्['वर्ण']}")
दर्शय(f"   Contains: {', '.join(इक्['वर्ण_सूची'])}")

# All consonants
हल् = प्रत्याहार_निष्कर्षण('ह', 'ल्')
दर्शय(f"\n   All consonants (हल्): {हल्['संख्या']} letters")

दर्शय("\n" + "=" * 60)
दर्शय("For detailed solutions, see the accompanying exercises notebook.")

---
## सन्दर्भ (References)

### प्राचीन ग्रन्थ:
1. **अष्टाध्यायी** - पाणिनि (c. 500 BCE)
2. **महाभाष्य** - पतञ्जलि (c. 150 BCE) - Commentary on Panini
3. **काशिका वृत्ति** - जयादित्य & वामन (c. 600 CE)
4. **सिद्धान्तकौमुदी** - भट्टोजि दीक्षित (c. 1600 CE)

### आधुनिक शोध:
1. Kiparsky, P. (1979). *Panini as a Variationist*. MIT Press.
2. Staal, J.F. (1988). *Universals: Studies in Indian Logic and Linguistics*. University of Chicago Press.
3. Cardona, G. (1997). *Pāṇini: His Work and its Traditions*. Motilal Banarsidass.
4. Ingerman, P.Z. (1967). "Panini-Backus Form". *Communications of the ACM* 10(3): 137.
5. Briggs, R. (1985). "Knowledge Representation in Sanskrit and Artificial Intelligence". *AI Magazine* 6(1): 32-39.

### ऑनलाइन संसाधन:
1. Sanskrit Heritage Platform: https://sanskrit.inria.fr/
2. Paninian Research Tool: https://panini.phil.hhu.de/
3. Digital Corpus of Sanskrit: http://www.sanskrit-linguistics.org/

### संगणकीय उपकरण:
1. Sanskrit Sandhi Splitter (Heritage Platform)
2. Morphological Analyzer (UoH)
3. Sanskrit WordNet

---

## Acknowledgments

यह नोटबुक पाणिनि के महान योगदान को आधुनिक संगणक विज्ञान के परिप्रेक्ष्य में समझने का प्रयास है।

**॥ तस्मै पाणिनये नमः ॥**

*Salutations to Panini, the first computational linguist.*